# 제주도 핫플레이스 - 인스타그램 크롤링

In [27]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [97]:
chromedriver = '/Users/hhlee/Desktop/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

### 1. Instagram에 접속하여 로그인 

In [98]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [99]:
with open('password.txt') as f:
        password = f.read()

In [100]:
email = 'i___n_ji'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [101]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [102]:
# 로그인 정보 저장 - 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [103]:
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주도맛집 으로 검색

In [104]:
from urllib.parse import quote
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [105]:
driver.get(url)
time.sleep(5)

In [106]:
# driver.close()

### 3. 첫번째 게시글 열기

In [107]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

### 4.  게시글 정보 가져오기

In [108]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [109]:
# 1) 본문 내용 가져오기
import unicodedata
try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
except:
        content = '  '
content

'.한림 #문도지오름방목된 조랑말과 교감까지 할 수이따규~일몰 시간에 맞춰서 가면 멋진 노을까지 볼 수 있으니 제주앓이에 무적권 걸린다...@이러니 제주를 사랑할 수 바꿰업쮜@코로롱시키!! 얼른 사라져!!✔장소_문도지오름✔주소_제주 제주시 한림읍 금악리-올레길 14-1코스에 위치-올레길가는법꿀팁_나뭇가지에 리본모양(파랑+빨강)끈이 묶여있다고한다_쭉가면 경고판문이 있는데 들어가면 된다_샛길로 5~6분정도 올라가면 된다..#제주맛집 #제주도맛집 #제주공항근처맛집 #제주시맛집 #애월맛집 #중문맛집 #제주공항맛집 #서귀포맛집 #산방산맛집 #신제주맛집 #제주시청맛집 #제주애월맛집 #월정리맛집 #제주여행 #제주핫플'

In [110]:
# 2) 본문 내용에서 해쉬태그 가져오기 '#[^\s#,\\]+'
import re
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#문도지오름방목된',
 '#제주맛집',
 '#제주도맛집',
 '#제주공항근처맛집',
 '#제주시맛집',
 '#애월맛집',
 '#중문맛집',
 '#제주공항맛집',
 '#서귀포맛집',
 '#산방산맛집',
 '#신제주맛집',
 '#제주시청맛집',
 '#제주애월맛집',
 '#월정리맛집',
 '#제주여행',
 '#제주핫플']

In [111]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-17'

In [112]:
# 4) 좋아요 수 가져오기 
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

0

In [113]:
# 5) 위치정보 가져오기
try:
     place = soup.select('div.M30cS')[0].text
     place = unicodedata.normalize('NFC', place)
except:
     place = ' '
place


''

In [114]:
row = [content, date, like, place, tags]
row

['.한림 #문도지오름방목된 조랑말과 교감까지 할 수이따규~일몰 시간에 맞춰서 가면 멋진 노을까지 볼 수 있으니 제주앓이에 무적권 걸린다...@이러니 제주를 사랑할 수 바꿰업쮜@코로롱시키!! 얼른 사라져!!✔장소_문도지오름✔주소_제주 제주시 한림읍 금악리-올레길 14-1코스에 위치-올레길가는법꿀팁_나뭇가지에 리본모양(파랑+빨강)끈이 묶여있다고한다_쭉가면 경고판문이 있는데 들어가면 된다_샛길로 5~6분정도 올라가면 된다..#제주맛집 #제주도맛집 #제주공항근처맛집 #제주시맛집 #애월맛집 #중문맛집 #제주공항맛집 #서귀포맛집 #산방산맛집 #신제주맛집 #제주시청맛집 #제주애월맛집 #월정리맛집 #제주여행 #제주핫플',
 '2021-08-17',
 0,
 '',
 ['#문도지오름방목된',
  '#제주맛집',
  '#제주도맛집',
  '#제주공항근처맛집',
  '#제주시맛집',
  '#애월맛집',
  '#중문맛집',
  '#제주공항맛집',
  '#서귀포맛집',
  '#산방산맛집',
  '#신제주맛집',
  '#제주시청맛집',
  '#제주애월맛집',
  '#월정리맛집',
  '#제주여행',
  '#제주핫플']]

### 5. 다음 게시글로 이동

In [115]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [121]:
# 6을 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 6. 여러개의 게시글 가져오기

In [123]:
from tqdm.notebook import tqdm

In [128]:
def get_content(driver):
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        try:
            content = soup.select_one('div.C4VMK > span').text
            content = unicodedata.normalize('NFC', content)
        except:
                content = '  '
        tags = re.findall(r'#[^\s#,\\]+', content)
        date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

        try:
            like = soup.select_one('div.Nm9Fw').text[4:-1]
        except:
            like = 0
        
        try:
            place = soup.select('div.M30cS')[0].text
            place = unicodedata.normalize('NFC', place)
        except:
            place = ' ' 

        return [content, tags, date, like, place]

In [122]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [129]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            

            

  0%|          | 0/500 [00:00<?, ?it/s]

In [140]:
df = pd.DataFrame(results, columns=['content','tags','date','like','place',])
df.head()

,content,tags,date,like,place
0,시댁 여자들만의 점심 회동이 있던날💚주한아빠 안테 주한이 맡겨두고 아침 일찍 달려 ...,"[#베케, #서귀포카페]",2021-08-18,823,Jeju
1,눈부시게 맑았던 오늘 하늘과 햇님 찰칵📸다음주 #제주여행 이예약되어 있는데제주도가 ...,"[#제주여행, #백신접종, #운좋은사람들, #운이좋아지는비누, #허브케어플러스, #...",2021-08-18,111,Suwon
2,.#연동 #모니카옛날통닭간판부터 맛집포스 작렬하는 찐맛집이젠 인생치킨이라고 말하기도...,"[#연동, #모니카옛날통닭간판부터, #제주맛집, #제주도맛집, #제주공항근처맛집, ...",2021-08-19,"1,124",
3,42년 전통 도민 맛집. 태양을 품은 듯 화끈한 비주얼이다. 포슬포슬한 두부가 푸짐...,"[#신해바라기분식, #제주도맛집뜨끈한, #순두부찌개, #국밥맛집, #제주맛집, #제...",2021-08-18,"1,081",제주도
4,#제주전복맛집JEJU하면 빼먹을 수 없는 전복😆이때 먹지 언제 먹어보겠어~존맛탱 전...,"[#제주전복맛집JEJU하면, #제주여행, #제주도여행, #제주여행코스, #제주가볼만...",2021-08-18,"1,270",


In [120]:
df.tail()

,content,date,like,place,tags
45,아보카도커피🥑☕아보카도샌드위치🥑🥪,2021-08-19,0,,[]
46,🌻나만 보는 해바라기 같은 남자 구합니다🌻...#일상 #일상스타그램 #인스타그램 #...,2021-08-19,0,,"[#일상, #일상스타그램, #인스타그램, #여행, #제주도, #제주도여행, #제주도..."
47,이제 슬슬 선선해지고 있다❤️,2021-08-19,0,Jeju-do,[]
48,오모나...#헬스 #국내여행 #내사랑 #제주도카페 #다렉 #새벽 #일산 #가족 #카...,2021-08-19,0,,"[#헬스, #국내여행, #내사랑, #제주도카페, #다렉, #새벽, #일산, #가족,..."
49,제주 인생밥집⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀🕚 OPEN - 11:00⠀⠀⠀⠀⠀⠀⠀⠀...,2021-08-19,0,,"[#제주, #제주도, #jeju, #제주살이, #제주맛집, #제주도맛집, #한림맛집..."


In [130]:
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(5)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(2)

    results = []
    for _ in tqdm(range(200)):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
        except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)

    df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
    df.to_csv(f'data/{keyword}.csv', index=False)

    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
    time.sleep(2)

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [131]:
driver.close()

### 4가지 CSV 파일의 중복을 제거한 후 통합 저장

In [142]:
for keyword in ['제주도맛집','제주맛집','제주도관광','제주여행']:
        df = pd.read_csv(f'data/{keyword}.csv')
        print(df.head())


                                             content  \
0  .#연동 #모니카옛날통닭간판부터 맛집포스 작렬하는 찐맛집이젠 인생치킨이라고 말하기도...   
1  <광고>섭지코지에서 한끼 맛있게 드시고 싶다면...특히 점심메뉴로 강력추천드리고요 ...   
2  .< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰...   
3  협찬제주애월맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 : 064-...   
4  .한림 #문도지오름방목된 조랑말과 교감까지 할 수이따규~일몰 시간에 맞춰서 가면 멋...   

                                                date        like  place tags  
0  ['#연동', '#모니카옛날통닭간판부터', '#제주맛집', '#제주도맛집', '#제...  2021-08-19  1,113  NaN  
1  ['#섭지코지한끼🍱오픈', '#섭지코지맛집', '#섭지코지', '#섭지코지카페', ...  2021-08-18      0  NaN  
2  ['#제주맛집', '#제주도맛집', '#제주공항근처맛집', '#제주시맛집', '#애...  2021-08-18  1,008  NaN  
3  ['#애월맛집', '#애월카페', '#애월읍맛집', '#제주애월맛집', '#제주도맛...  2021-08-18      0  NaN  
4  ['#문도지오름방목된', '#제주맛집', '#제주도맛집', '#제주공항근처맛집', ...  2021-08-17  1,304  NaN  
                                             content  \
0  .💙 구좌 "제주모랑" 💙사장님들이 손수 만든 수제소품들이 가득~좋은향이 가득한 구...   
1  협찬제주 대표 썬셋 맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 :...   
2  42년 전통 도민 맛집. 태양을 

In [137]:
jeju_df.tail()

""
